<a href="https://colab.research.google.com/github/kuroneko913/lab/blob/master/RSA%E6%9A%97%E5%8F%B7%E3%82%92%E3%82%84%E3%81%A3%E3%81%A6%E3%81%BF%E3%81%9F(%E5%8B%95%E4%BD%9C%E7%A2%BA%E8%AA%8D%E6%B8%88%E3%81%BF).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RSA暗号 を実装する

In [0]:
from numpy import random
import math
import base64

In [0]:
# ax+by=gcd(a,b)となるx,y,gcd(a,b)を求める(拡張ユークリッドの互除法)
def ext_euclid_gcd(z1,z2,x1=1,y1=0,x2=0,y2=1):
  if z2==0: return x1,y1,z1
  q = z1//z2
  x = x1 - q * x2
  y = y1 - q * y2
  z = z1 - q * z2
  return ext_euclid_gcd(z2,z,x2,y2,x,y)

In [0]:
# a,bの最小公倍数はa*b/gcd(a,b)
def lcm(a,b):
  return a*b/ext_euclid_gcd(a,b)[-1]

In [0]:
# 素数判定法のアルゴリズム
def is_prime(n:int)->bool:
  limit = int(math.sqrt(n))+1
  if n == 2: return True
  if n==0 or n == 1 or n%2==0 : return False
  for i in range(3,limit,2):
    if n%i==0: return False
  return True

In [0]:
# ユークリッドの互除法により、最大公約数(GCD)を求める
def euclid_r_gcd(a,b):
  if (a>b):
    n = a
    m = b
  else:
    n = b
    m = a
  if (m==0): return n
  else:
    print(f'{n}%{m}:{n%m},{n}-{int(n/m)}*{m}={n%m}')
    return euclid_r_gcd(m,n%m)

In [7]:
euclid_r_gcd(7,5)

7%5:2,7-1*5=2
5%2:1,5-2*2=1
2%1:0,2-2*1=0


1

In [8]:
ext_euclid_gcd(84,11)

(-3, 23, 1)

In [9]:
print(lcm(5,32))
print(ext_euclid_gcd(5,32))

160.0
(13, -2, 1)


公開鍵と秘密鍵を生成する

In [24]:
# 巨大な素数の組の生成
gcd = 10
num_length = 14 # 計算精度の問題か、20以上だとうまく復号化できないことがある
prime_p = False
prime_q = False
count = 0
while (gcd !=1 or prime_p is False or prime_q is False):
  count += 1
  print(f'try count:{count}')
  if (prime_p is False ): 
    p = random.randint(2**num_length/2)
    prime_p = is_prime(p)
  if (prime_q is False): 
    q = random.randint(2**num_length/2)
    prime_q = is_prime(q)
  gcd = ext_euclid_gcd(p,q)[-1]
  

try count:1


In [25]:
# 公開鍵
e = 65537
n = p*q
print(e,n)

65537 51271247


In [26]:
print(p,q)
is_prime(q)

6703 7649


True

In [27]:
# 秘密鍵を生成する
phi_m = lcm(p-1,q-1)
ext_euclid_gcd_result = ext_euclid_gcd(e,phi_m)
d = abs(int(ext_euclid_gcd_result[0]))
print(abs(d))

8216417


In [0]:
# 暗号化 & 復号化
plain_text = 'Hello,RSA!'
ord_plain_text = list(map(ord,plain_text))
encode = lambda x,e,n: list(map(lambda value: pow(value,e,n),x))

In [29]:
# 暗号化
print(ord_plain_text)
codes = encode(ord_plain_text,e,n)
#print(''.join(list(map(chr,codes))))
print(codes)

[72, 101, 108, 108, 111, 44, 82, 83, 65, 33]
[13044257, 28656678, 38375306, 38375306, 36754685, 21387370, 4868034, 9640745, 47387616, 46032559]


In [30]:
# 復号化
decodes = encode(codes, d, n)
#print(''.join(list(map(chr,decodes))))
print(decodes)
print(''.join(list(map(chr,decodes))))

[72, 101, 108, 108, 111, 44, 82, 83, 65, 33]
Hello,RSA!


In [31]:
# デジタル署名もやってみる
m = "This messsage is written by myblackcat7112 2019.08.29 08:50 "
ord_m = list(map(ord,m))
print(ord_m)

[84, 104, 105, 115, 32, 109, 101, 115, 115, 115, 97, 103, 101, 32, 105, 115, 32, 119, 114, 105, 116, 116, 101, 110, 32, 98, 121, 32, 109, 121, 98, 108, 97, 99, 107, 99, 97, 116, 55, 49, 49, 50, 32, 50, 48, 49, 57, 46, 48, 56, 46, 50, 57, 32, 48, 56, 58, 53, 48, 32]


In [32]:
# 自分の持っている秘密鍵で暗号化
encoded_message = encode(ord_m,d,n)
print(' '.join(map(str,encoded_message)))

45196586 5891317 42966743 43087734 20177486 13821140 10002532 43087734 43087734 43087734 17645678 39501014 10002532 20177486 42966743 43087734 20177486 27167041 12993305 42966743 46887186 46887186 10002532 5854637 20177486 13550209 36494385 20177486 13821140 36494385 13550209 19945716 17645678 1448087 32790184 1448087 17645678 46887186 9146282 4794898 4794898 41748546 20177486 41748546 40283795 4794898 1406546 3663245 40283795 28006695 3663245 41748546 1406546 20177486 40283795 28006695 22168417 34695713 40283795 20177486


In [37]:
# 公開鍵で復号するとメッセージが読める
decoded_message = encode(encoded_message,e,n)
print(''.join(list(map(chr,decoded_message))))

This messsage is written by myblackcat7112 2019.08.29 08:50 


In [34]:
e,n

(65537, 51271247)

In [35]:
# せっかくなので、base64に変換して、それっぽくする
str_message = ' '.join(map(str,encoded_message)).encode('utf-8')
base64_coded = base64.b64encode(str_message)
v_ = ''
v__ = []
for v in base64_coded:
  if (len(v_)==76):
    v__.append(v_)
    v_ =''
  v_ += chr(v)
v__.append(v_)
for v in v__:
  print(v)

NDUxOTY1ODYgNTg5MTMxNyA0Mjk2Njc0MyA0MzA4NzczNCAyMDE3NzQ4NiAxMzgyMTE0MCAxMDAw
MjUzMiA0MzA4NzczNCA0MzA4NzczNCA0MzA4NzczNCAxNzY0NTY3OCAzOTUwMTAxNCAxMDAwMjUz
MiAyMDE3NzQ4NiA0Mjk2Njc0MyA0MzA4NzczNCAyMDE3NzQ4NiAyNzE2NzA0MSAxMjk5MzMwNSA0
Mjk2Njc0MyA0Njg4NzE4NiA0Njg4NzE4NiAxMDAwMjUzMiA1ODU0NjM3IDIwMTc3NDg2IDEzNTUw
MjA5IDM2NDk0Mzg1IDIwMTc3NDg2IDEzODIxMTQwIDM2NDk0Mzg1IDEzNTUwMjA5IDE5OTQ1NzE2
IDE3NjQ1Njc4IDE0NDgwODcgMzI3OTAxODQgMTQ0ODA4NyAxNzY0NTY3OCA0Njg4NzE4NiA5MTQ2
MjgyIDQ3OTQ4OTggNDc5NDg5OCA0MTc0ODU0NiAyMDE3NzQ4NiA0MTc0ODU0NiA0MDI4Mzc5NSA0
Nzk0ODk4IDE0MDY1NDYgMzY2MzI0NSA0MDI4Mzc5NSAyODAwNjY5NSAzNjYzMjQ1IDQxNzQ4NTQ2
IDE0MDY1NDYgMjAxNzc0ODYgNDAyODM3OTUgMjgwMDY2OTUgMjIxNjg0MTcgMzQ2OTU3MTMgNDAy
ODM3OTUgMjAxNzc0ODY=


In [36]:
# base64をdecodeしてから、デジタル署名メッセージが復元されることを確認
sign_message=''.join(v__)
for v in v__:
  print(v)
base64_decoded_codes = map(int,base64.b64decode(sign_message).decode('utf-8').split(' '))
# メッセージと公開鍵さえあれば、復元できる！
#e,n = 65537, 2341789
print(base64_decoded_codes)
''.join(map(chr,encode(base64_decoded_codes,e,n)))

NDUxOTY1ODYgNTg5MTMxNyA0Mjk2Njc0MyA0MzA4NzczNCAyMDE3NzQ4NiAxMzgyMTE0MCAxMDAw
MjUzMiA0MzA4NzczNCA0MzA4NzczNCA0MzA4NzczNCAxNzY0NTY3OCAzOTUwMTAxNCAxMDAwMjUz
MiAyMDE3NzQ4NiA0Mjk2Njc0MyA0MzA4NzczNCAyMDE3NzQ4NiAyNzE2NzA0MSAxMjk5MzMwNSA0
Mjk2Njc0MyA0Njg4NzE4NiA0Njg4NzE4NiAxMDAwMjUzMiA1ODU0NjM3IDIwMTc3NDg2IDEzNTUw
MjA5IDM2NDk0Mzg1IDIwMTc3NDg2IDEzODIxMTQwIDM2NDk0Mzg1IDEzNTUwMjA5IDE5OTQ1NzE2
IDE3NjQ1Njc4IDE0NDgwODcgMzI3OTAxODQgMTQ0ODA4NyAxNzY0NTY3OCA0Njg4NzE4NiA5MTQ2
MjgyIDQ3OTQ4OTggNDc5NDg5OCA0MTc0ODU0NiAyMDE3NzQ4NiA0MTc0ODU0NiA0MDI4Mzc5NSA0
Nzk0ODk4IDE0MDY1NDYgMzY2MzI0NSA0MDI4Mzc5NSAyODAwNjY5NSAzNjYzMjQ1IDQxNzQ4NTQ2
IDE0MDY1NDYgMjAxNzc0ODYgNDAyODM3OTUgMjgwMDY2OTUgMjIxNjg0MTcgMzQ2OTU3MTMgNDAy
ODM3OTUgMjAxNzc0ODY=


'This messsage is written by myblackcat7112 2019.08.29 08:50 '